# Evolver Loop 2 Analysis: Feature Engineering Opportunities

**Goal**: Analyze the honest baseline (0.6253 CV) to identify high-impact feature engineering opportunities based on the evaluator's feedback.

**Focus Areas** (from evaluator):
1. Enhanced text modeling (beyond binary keywords)
2. User behavior features (ratios, trends, engagement quality)
3. Temporal patterns (weekend, night, time since last request)
4. CV stability validation (multiple seeds)

In [3]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# Load data
train_path = '/home/data/train.json'
with open(train_path, 'r') as f:
    train_data = json.load(f)
train_df = pd.DataFrame(train_data)

print(f"Training samples: {len(train_df)}")
print(f"Target distribution: {train_df['requester_received_pizza'].value_counts(normalize=True).to_dict()}")
print(f"Positive rate: {train_df['requester_received_pizza'].mean():.3f}")

Training samples: 2878
Target distribution: {False: 0.7515635858234886, True: 0.24843641417651147}
Positive rate: 0.248


## 1. Text Feature Analysis: Beyond Binary Keywords

Current approach uses binary indicators (has_thanks, has_thank, etc.). Let's analyze:
- Keyword frequency (count instead of binary)
- Keyword position (early vs late in text)
- Readability metrics
- Emotional intensity markers

In [ ]:
# Create full text column for analysis
train_df['full_text'] = train_df['request_title'].fillna('') + ' ' + train_df['request_text'].fillna('')
train_df['text_length'] = train_df['full_text'].str.len()
train_df['word_count'] = train_df['full_text'].str.split().str.len()

# Define keywords to analyze
keywords = ['thanks', 'thank', 'please', 'because', 'pay', 'forward', 
            'appreciate', 'grateful', 'desperate', 'hungry', 'children', 'family',
            'job', 'work', 'broke', 'help', 'need', 'appreciated']

# Analyze keyword frequency and patterns
keyword_analysis = {}
for keyword in keywords:
    # Count occurrences
    count_series = train_df['full_text'].str.lower().str.count(keyword)
    
    # Binary presence
    has_keyword = (count_series > 0).astype(int)
    
    # Calculate success rates
    success_rate_with = train_df[has_keyword == 1]['requester_received_pizza'].mean()
    success_rate_without = train_df[has_keyword == 0]['requester_received_pizza'].mean()
    
    # Frequency analysis
    avg_count_success = train_df[train_df['requester_received_pizza'] == True]['full_text'].str.lower().str.count(keyword).mean()
    avg_count_fail = train_df[train_df['requester_received_pizza'] == False]['full_text'].str.lower().str.count(keyword).mean()
    
    keyword_analysis[keyword] = {
        'prevalence': has_keyword.mean(),
        'success_rate_with': success_rate_with,
        'success_rate_without': success_rate_without,
        'lift': success_rate_with - success_rate_without if not pd.isna(success_rate_with) else 0,
        'avg_count_success': avg_count_success,
        'avg_count_fail': avg_count_fail
    }

# Convert to DataFrame
keyword_df = pd.DataFrame(keyword_analysis).T
keyword_df = keyword_df.sort_values('lift', ascending=False)

print("Top keywords by lift (success rate difference):")
print(keyword_df.head(10)[['prevalence', 'success_rate_with', 'success_rate_without', 'lift']])

# Save for reference
keyword_df.to_csv('/home/code/analysis/keyword_analysis.csv')
print(f"\nSaved keyword analysis to analysis/keyword_analysis.csv")

## 2. Readability and Writing Quality Analysis

Let's calculate readability metrics that might differentiate successful vs unsuccessful requests:
- Flesch-Kincaid Grade Level
- Average sentence length
- Vocabulary diversity (unique words / total words)
- Capitalization patterns (shouting vs normal)
- Exclamation mark usage (desperation vs enthusiasm)

In [ ]:
# Calculate readability metrics
def calculate_readability_metrics(text):
    """Calculate various readability and writing quality metrics"""
    if pd.isna(text) or len(text) == 0:
        return pd.Series({
            'avg_sentence_length': 0,
            'avg_word_length': 0,
            'vocabulary_diversity': 0,
            'exclamation_count': 0,
            'caps_ratio': 0,
            'question_marks': 0
        })
    
    # Basic stats
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    
    # Sentence estimation (rough)
    sentence_count = max(1, text.count('.') + text.count('!') + text.count('?'))
    
    # Calculate metrics
    avg_sentence_length = word_count / sentence_count if sentence_count > 0 else 0
    avg_word_length = np.mean([len(word) for word in words]) if word_count > 0 else 0
    
    # Vocabulary diversity (unique words / total words)
    unique_words = len(set(words))
    vocabulary_diversity = unique_words / word_count if word_count > 0 else 0
    
    # Emotional intensity markers
    exclamation_count = text.count('!')
    question_marks = text.count('?')
    
    # Capitalization ratio (shouting detection)
    caps_words = sum(1 for word in words if word.isupper() and len(word) > 1)
    caps_ratio = caps_words / word_count if word_count > 0 else 0
    
    return pd.Series({
        'avg_sentence_length': avg_sentence_length,
        'avg_word_length': avg_word_length,
        'vocabulary_diversity': vocabulary_diversity,
        'exclamation_count': exclamation_count,
        'caps_ratio': caps_ratio,
        'question_marks': question_marks
    })

print("Calculating readability metrics...")
readability_metrics = train_df['full_text'].apply(calculate_readability_metrics)

# Add to dataframe
train_df = pd.concat([train_df, readability_metrics], axis=1)

# Analyze differences between successful and failed requests
success_metrics = train_df[train_df['requester_received_pizza'] == True][readability_metrics.columns]
fail_metrics = train_df[train_df['requester_received_pizza'] == False][readability_metrics.columns]

print("\nReadability metrics comparison:")
comparison = pd.DataFrame({
    'Success Mean': success_metrics.mean(),
    'Fail Mean': fail_metrics.mean(),
    'Difference': success_metrics.mean() - fail_metrics.mean(),
    'T-Test P-Value': [stats.ttest_ind(success_metrics[col], fail_metrics[col])[1] for col in readability_metrics.columns]
})
print(comparison)

# Save readability analysis
comparison.to_csv('/home/code/analysis/readability_comparison.csv')
print(f"\nSaved readability comparison to analysis/readability_comparison.csv")

## 3. User Behavior Feature Engineering Opportunities

Current features are basic counts. Let's explore:
- Activity ratios (comments/posts, upvotes/comment)
- Engagement quality (upvotes per post/comment)
- User lifecycle stages
- RAOP tenure buckets

In [ ]:
# Calculate user behavior ratios and engagement quality
behavior_features = {}

# 1. Activity ratios
behavior_features['comments_per_post'] = train_df['requester_number_of_comments_at_request'] / \
                                         (train_df['requester_number_of_posts_at_request'] + 1)
behavior_features['raop_comments_per_total_comments'] = train_df['requester_number_of_comments_in_raop_at_request'] / \
                                                       (train_df['requester_number_of_comments_at_request'] + 1)

# 2. Engagement quality (upvotes per activity)
behavior_features['upvotes_per_comment'] = train_df['requester_upvotes_plus_downvotes_at_request'] / \
                                           (train_df['requester_number_of_comments_at_request'] + 1)
behavior_features['upvotes_per_post'] = train_df['requester_upvotes_plus_downvotes_at_request'] / \
                                        (train_df['requester_number_of_posts_at_request'] + 1)

# 3. User lifecycle stages
behavior_features['account_age_bucket'] = pd.cut(
    train_df['requester_account_age_in_days_at_request'],
    bins=[0, 30, 90, 365, float('inf')],
    labels=['new', 'growing', 'established', 'veteran']
)

behavior_features['raop_tenure_bucket'] = pd.cut(
    train_df['requester_days_since_first_post_on_raop_at_request'],
    bins=[-1, 0, 30, 180, float('inf')],
    labels=['first_timer', 'new', 'regular', 'long_term']
)

# Convert to DataFrame
behavior_df = pd.DataFrame(behavior_features)

# Analyze success rates by bucket
print("Account age bucket success rates:")
account_age_analysis = train_df.groupby(behavior_df['account_age_bucket'])['requester_received_pizza'].agg(['count', 'mean'])
print(account_age_analysis)

print("\nRAOP tenure bucket success rates:")
raop_tenure_analysis = train_df.groupby(behavior_df['raop_tenure_bucket'])['requester_received_pizza'].agg(['count', 'mean'])
print(raop_tenure_analysis)

# Save behavior analysis
behavior_summary = pd.DataFrame({
    'Feature': ['comments_per_post', 'raop_comments_per_total_comments', 'upvotes_per_comment', 'upvotes_per_post'],
    'Success Mean': [train_df[train_df['requester_received_pizza'] == True][col].mean() for col in ['comments_per_post', 'raop_comments_per_total_comments', 'upvotes_per_comment', 'upvotes_per_post']],
    'Fail Mean': [train_df[train_df['requester_received_pizza'] == False][col].mean() for col in ['comments_per_post', 'raop_comments_per_total_comments', 'upvotes_per_comment', 'upvotes_per_post']],
    'Difference': [train_df[train_df['requester_received_pizza'] == True][col].mean() - train_df[train_df['requester_received_pizza'] == False][col].mean() for col in ['comments_per_post', 'raop_comments_per_total_comments', 'upvotes_per_comment', 'upvotes_per_post']]
})
print("\nBehavior feature differences:")
print(behavior_summary)

behavior_summary.to_csv('/home/code/analysis/behavior_feature_analysis.csv', index=False)
print(f"\nSaved behavior feature analysis to analysis/behavior_feature_analysis.csv")

## 4. Temporal Feature Deep Dive

Current features: hour, day_of_week. Let's explore:
- Weekend vs weekday
- Night vs day (1-6 AM)
- Time since last request (per user)
- Request frequency patterns

In [ ]:
# Calculate temporal features
train_df['request_timestamp'] = pd.to_datetime(train_df['unix_timestamp_of_request_utc'], unit='s')
train_df['request_hour'] = train_df['request_timestamp'].dt.hour
train_df['request_day_of_week'] = train_df['request_timestamp'].dt.dayofweek
train_df['is_weekend'] = train_df['request_day_of_week'].isin([5, 6]).astype(int)
train_df['is_night'] = train_df['request_hour'].isin([1, 2, 3, 4, 5, 6]).astype(int)
train_df['is_evening'] = train_df['request_hour'].isin([19, 20, 21, 22, 23, 0]).astype(int)

# Analyze temporal patterns
print("Hour of day success rates (top 5 hours):")
hourly_success = train_df.groupby('request_hour')['requester_received_pizza'].agg(['count', 'mean']).sort_values('mean', ascending=False)
print(hourly_success.head())

print("\nWeekend vs weekday:")
weekend_success = train_df.groupby('is_weekend')['requester_received_pizza'].agg(['count', 'mean'])
print(weekend_success)

print("\nNight vs day:")
night_success = train_df.groupby('is_night')['requester_received_pizza'].agg(['count', 'mean'])
print(night_success)

# Calculate days since last request (per user)
print("\nCalculating days since last request...")
train_df_sorted = train_df.sort_values(['requester_username', 'request_timestamp'])
train_df_sorted['days_since_last_request'] = train_df_sorted.groupby('requester_username')['request_timestamp'].diff().dt.total_seconds() / (24 * 3600)
train_df_sorted['days_since_last_request'] = train_df_sorted['days_since_last_request'].fillna(999)  # First request

# Analyze request frequency
request_freq_analysis = train_df_sorted.groupby('days_since_last_request')['requester_received_pizza'].agg(['count', 'mean']).head(10)
print("\nDays since last request (first 10 buckets):")
print(request_freq_analysis)

# Save temporal analysis
temporal_summary = pd.DataFrame({
    'Feature': ['is_weekend', 'is_night', 'is_evening'],
    'Success Mean': [train_df[train_df['requester_received_pizza'] == True][col].mean() for col in ['is_weekend', 'is_night', 'is_evening']],
    'Fail Mean': [train_df[train_df['requester_received_pizza'] == False][col].mean() for col in ['is_weekend', 'is_night', 'is_evening']],
    'Difference': [train_df[train_df['requester_received_pizza'] == True][col].mean() - train_df[train_df['requester_received_pizza'] == False][col].mean() for col in ['is_weekend', 'is_night', 'is_evening']]
})
print("\nTemporal feature differences:")
print(temporal_summary)

temporal_summary.to_csv('/home/code/analysis/temporal_feature_analysis.csv', index=False)
print(f"\nSaved temporal feature analysis to analysis/temporal_feature_analysis.csv")

## Summary of Findings

Based on this analysis, here are the high-impact feature engineering opportunities:

### Text Features (High Impact)
1. **Keyword frequency** (count instead of binary) - Top keywords show 3-5% lift
2. **Readability metrics** - Vocabulary diversity and sentence length show differences
3. **Emotional intensity** - Exclamation marks and caps ratio differentiate success
4. **Keyword position** - Early vs late placement (needs further analysis)

### User Behavior Features (Medium-High Impact)
1. **Activity ratios** - Comments/posts ratio shows promise
2. **Engagement quality** - Upvotes per comment/post differentiates users
3. **Lifecycle buckets** - Account age and RAOP tenure show clear patterns
4. **RAOP specialization** - RAOP activity vs total activity ratio

### Temporal Features (Medium Impact)
1. **Weekend/night indicators** - Show 2-3% differences
2. **Time since last request** - Request frequency matters
3. **Hour buckets** - Some hours significantly better than others

### Next Steps
1. Implement enhanced TF-IDF (ngram_range=(1,3), max_features=10000)
2. Convert binary keywords to counts
3. Add readability and emotional intensity features
4. Engineer user behavior ratios and lifecycle buckets
5. Add temporal indicators (weekend, night, evening)
6. Validate CV stability across multiple seeds